In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Load the Cleveland Heart Disease dataset
cleveland_data = pd.read_csv('Heart_disease_cleveland_new.csv', names=[
    "age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak",
    "slope", "ca", "thal", "target"
], na_values='?')

# Load the Statlog (Heart) dataset from local file
statlog_file_path = 'heart.dat'
statlog_column_names = [
    "age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak",
    "slope", "ca", "thal", "target"
]
statlog_data = pd.read_csv(statlog_file_path, names=statlog_column_names, delim_whitespace=True)

# Adjust 'slope' and 'thal' values in Statlog dataset to match Cleveland dataset
statlog_data['slope'] = statlog_data['slope'].replace({1: 1, 2: 2, 3: 0})
statlog_data['thal'] = statlog_data['thal'].replace({3: 1, 6: 2, 7: 3})
statlog_data['target'] = statlog_data['target'].replace({1: 0, 2: 1})

# Combine the datasets
data = pd.concat([cleveland_data, statlog_data])
# Convert target column to numeric and handle missing values
data['target'] = pd.to_numeric(data['target'], errors='coerce')
data.dropna(inplace=True)
data['target'] = data['target'].astype(int)

# Split the data into features and target
X = data.drop('target', axis=1)
y = data['target'].apply(lambda x: 1 if x > 0 else 0)  # Convert target to binary

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

def create_ann_model(input_shape, layer_sizes, activation='relu', output_activation='sigmoid'):
    model = Sequential()
    model.add(tf.keras.Input(shape=(input_shape,)))
    for size in layer_sizes:
        model.add(Dense(size, activation=activation))
    model.add(Dense(1, activation=output_activation))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Define the layer sizes manually (you can adjust these values as needed)
layer_sizes = [100, 100]

# Train the ANN model with the defined parameters
best_model = create_ann_model(X_train.shape[1], layer_sizes)
best_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Evaluate the model
y_pred = (best_model.predict(X_test) > 0.5).astype("int32")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1 Score: {f1_score(y_test, y_pred)}")
print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred)}")

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Append the results to the CSV file
results = pd.DataFrame({
    'Model': ['Neural Network Traditional'],
    'Accuracy': [accuracy],
    'Precision': [precision],
    'Recall': [recall],
    'F1-Score': [f1]
})

results.to_csv('model_results.csv', mode='a', header=False, index=False)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Accuracy: 0.8434782608695652
Precision: 0.8333333333333334
Recall: 0.8
F1 Score: 0.8163265306122449
ROC AUC Score: 0.8384615384615385
